In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import time
from tqdm.notebook import tqdm
import torch.nn.functional as F
import sklearn

In [ ]:
class ANN_Model(nn.Module):
    def __init__(self, 
                 input_features = 16, 
                 out_features=2,
                 hdn_dim=30):
       
        super().__init__()
       
        self.linear_relu_stack = nn.Sequential( 
            nn.Linear(input_features, hdn_dim),
            nn.Dropout(p=0.3),
            nn.ReLU(),
            nn.Linear(hdn_dim, hdn_dim),
            nn.Dropout(p=0.3),
            nn.ReLU(),
            nn.Linear(hdn_dim, hdn_dim),
            nn.Dropout(p=0.3),
            nn.ReLU(),
            nn.Linear(hdn_dim, hdn_dim),
            nn.Dropout(p=0.3),
            nn.ReLU(),
            nn.Linear(hdn_dim, out_features),
        )

    def forward(self, x):
        out = self.linear_relu_stack(x)
        return out

In [ ]:
# Function to check device availability
def device_available(force_cpu=False): 
    if force_cpu or not torch.cuda.is_available():
        device = torch.device('cpu')
        print("Using CPU")
    else:
        device = torch.device('cuda')
        print("Using GPU")
    return device

In [ ]:
def evaluate(model, test_loader, device):
    #VALIDATION
    model.eval()
    test_preds, test_targets = [], []
    with torch.no_grad(): #to not calculate and update gradients since its a validation
        for inpt, target in test_loader:
            inpt, target = inpt.to(device), target.to(device)
            out = model(inpt)
            _, preds = torch.max(out, dim=1) 
            test_preds.extend(preds.cpu().numpy())
            test_targets.extend(target.cpu().numpy())
    
    #METRICS
    accuracy = accuracy_score(test_targets, test_preds)
    precision_pos = precision_score(test_targets, test_preds, pos_label=1)  
    precision_neg = precision_score(test_targets, test_preds, pos_label=0)
    return accuracy, precision_pos, precision_neg

In [ ]:
model = ANN_Model()

model.load_state_dict(torch.load('model_features.pth', weights_only=True))

device = device_available(force_cpu=True)

In [ ]:
model.eval()  # This disables dropout/batch norm if you have any


X_test = np.load("X_test.npy")
Y_test = np.load("Y_test.npy")

X_test = torch.tensor(X_test)
Y_test = torch.tensor(Y_test)
test_dataset = TensorDataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, 
                            batch_size=64,
                            shuffle=False,
                            num_workers=4,
                            pin_memory=True)

# Run evaluation using your existing function
accuracy, precision_pos, precision_neg = evaluate(model, test_loader, device)

# Print results
print(f"Accuracy: {accuracy}")
print(f"Positive Precision: {precision_pos}")
print(f"Negative Precision: {precision_neg}")